# citation for noise methods: https://github.com/hendrycks/robustness/blob/master/ImageNet-C/create_c/make_cifar_c.py

https://stackoverflow.com/questions/22937589/how-to-add-noise-gaussian-salt-and-pepper-etc-to-image-in-python-with-opencv

https://www.geeksforgeeks.org/opencv-motion-blur-in-python/

In [1]:
import cv2 
import numpy as np
import torch
import matplotlib.pyplot as plt

In [2]:
train_x = torch.load('train_x.pt', map_location=torch.device('cpu'))
train_y = torch.load('train_y.pt', map_location=torch.device('cpu'))
test_x = torch.load('test_x.pt', map_location=torch.device('cpu'))
test_y = torch.load('test_y.pt', map_location=torch.device('cpu'))

In [3]:
print("train_x shape: ", train_x.size())
print("train_y shape: ", train_y.size())
print("test_x shape: ", test_x.size())
print("test_y shape: ", test_y.size())

train_x shape:  torch.Size([39209, 50, 50, 3])
train_y shape:  torch.Size([39209])
test_x shape:  torch.Size([12630, 50, 50, 3])
test_y shape:  torch.Size([12630])


In [4]:
train_x = train_x.numpy().astype('float32')
train_y = train_y.numpy().astype('float32')
test_x = test_x.numpy().astype('float32')
test_y = test_y.numpy().astype('float32')

In [5]:
print("train_x shape: ", train_x.shape)
print("train_y shape: ", train_y.shape)
print("test_x shape: ", test_x.shape)
print("test_y shape: ", test_y.shape)

train_x shape:  (39209, 50, 50, 3)
train_y shape:  (39209,)
test_x shape:  (12630, 50, 50, 3)
test_y shape:  (12630,)


In [6]:
idx = np.random.permutation(train_x.shape[0])
train_x_shuffle, train_y_shuffle = train_x[idx], train_y[idx]
idx = np.random.permutation(test_x.shape[0])
test_x_shuffle, test_y_shuffle = test_x[idx], test_y[idx]

In [7]:
train_x_shuffle = np.swapaxes(train_x_shuffle,3,1)
test_x_shuffle = np.swapaxes(test_x_shuffle,3,1)
train_x_shuffle = np.swapaxes(train_x_shuffle,2,3)
test_x_shuffle = np.swapaxes(test_x_shuffle,2,3)

In [8]:
val_size = int(train_x.shape[0] * 0.2)
val_x = train_x_shuffle[0:val_size]
val_y = train_y_shuffle[0:val_size]
train_x_shuffle = train_x_shuffle[val_size:]
train_y_shuffle = train_y_shuffle[val_size:]

In [9]:
train_x = torch.from_numpy(train_x_shuffle)
train_y = torch.from_numpy(train_y_shuffle)
val_x = torch.from_numpy(val_x)
val_y = torch.from_numpy(val_y)
test_x = torch.from_numpy(test_x_shuffle)
test_y = torch.from_numpy(test_y_shuffle)

In [10]:
train_x = train_x.numpy().astype('float32')
train_y = train_y.numpy().astype('float32')
val_x = val_x.numpy().astype('float32')
val_y = val_y.numpy().astype('float32')
test_x = test_x.numpy().astype('float32')
test_y = test_y.numpy().astype('float32')

In [11]:
print("train_x shape: ", train_x.shape)
print("train_y shape: ", train_y.shape)
print("val_x shape: ", val_x.shape)
print("val_y shape: ", val_y.shape)
print("test_x shape: ", test_x.shape)
print("test_y shape: ", test_y.shape)

train_x shape:  (31368, 3, 50, 50)
train_y shape:  (31368,)
val_x shape:  (7841, 3, 50, 50)
val_y shape:  (7841,)
test_x shape:  (12630, 3, 50, 50)
test_y shape:  (12630,)


In [12]:
train_x_rotate = np.zeros((train_x.shape[0] * 4, train_x.shape[1], train_x.shape[2], train_x.shape[3]))
train_y_rotate = np.zeros((train_y.shape[0] * 4, 2))
val_x_rotate = np.zeros((val_x.shape[0] * 4, val_x.shape[1], val_x.shape[2], val_x.shape[3]))
val_y_rotate = np.zeros((val_y.shape[0] * 4, 2))
test_x_rotate = np.zeros((test_x.shape[0] * 4, test_x.shape[1], test_x.shape[2], test_x.shape[3]))
test_y_rotate = np.zeros((test_y.shape[0] * 4, 2))

In [13]:
print("train_x_rotate shape: ", train_x_rotate.shape)
print("train_y_rotate shape: ", train_y_rotate.shape)
print("val_x_rotate shape: ", val_x_rotate.shape)
print("val_y_rotate shape: ", val_y_rotate.shape)
print("test_x_rotate shape: ", test_x_rotate.shape)
print("test_y_rotate shape: ", test_y_rotate.shape)

train_x_rotate shape:  (125472, 3, 50, 50)
train_y_rotate shape:  (125472, 2)
val_x_rotate shape:  (31364, 3, 50, 50)
val_y_rotate shape:  (31364, 2)
test_x_rotate shape:  (50520, 3, 50, 50)
test_y_rotate shape:  (50520, 2)


In [14]:
for i in range(train_x.shape[0]):
    train_x_rotate[i * 4 + 0] = train_x[i]
    train_y_rotate[i * 4 + 0][0] = train_y[i]
    train_y_rotate[i * 4 + 0][1] = 0
    
    train_x_rotate[i * 4 + 1][0] = np.rot90(train_x[i][0])
    train_x_rotate[i * 4 + 1][1] = np.rot90(train_x[i][1])
    train_x_rotate[i * 4 + 1][2] = np.rot90(train_x[i][2])
    train_y_rotate[i * 4 + 1][0] = train_y[i]
    train_y_rotate[i * 4 + 1][1] = 1
    
    train_x_rotate[i * 4 + 2][0] = np.rot90(np.rot90(train_x[i][0]))
    train_x_rotate[i * 4 + 2][1] = np.rot90(np.rot90(train_x[i][1]))
    train_x_rotate[i * 4 + 2][2] = np.rot90(np.rot90(train_x[i][2]))
    train_y_rotate[i * 4 + 2][0] = train_y[i]
    train_y_rotate[i * 4 + 2][1] = 2
    
    train_x_rotate[i * 4 + 3][0] = np.rot90(np.rot90(np.rot90(train_x[i][0])))
    train_x_rotate[i * 4 + 3][1] = np.rot90(np.rot90(np.rot90(train_x[i][1])))
    train_x_rotate[i * 4 + 3][2] = np.rot90(np.rot90(np.rot90(train_x[i][2])))
    train_y_rotate[i * 4 + 3][0] = train_y[i]
    train_y_rotate[i * 4 + 3][1] = 3
    
for i in range(val_x.shape[0]):
    val_x_rotate[i * 4 + 0] = val_x[i]
    val_y_rotate[i * 4 + 0][0] = val_y[i]
    val_y_rotate[i * 4 + 0][1] = 0
    
    val_x_rotate[i * 4 + 1][0] = np.rot90(val_x[i][0])
    val_x_rotate[i * 4 + 1][1] = np.rot90(val_x[i][1])
    val_x_rotate[i * 4 + 1][2] = np.rot90(val_x[i][2])
    val_y_rotate[i * 4 + 1][0] = val_y[i]
    val_y_rotate[i * 4 + 1][1] = 1
    
    val_x_rotate[i * 4 + 2][0] = np.rot90(np.rot90(val_x[i][0]))
    val_x_rotate[i * 4 + 2][1] = np.rot90(np.rot90(val_x[i][1]))
    val_x_rotate[i * 4 + 2][2] = np.rot90(np.rot90(val_x[i][2]))
    val_y_rotate[i * 4 + 2][0] = val_y[i]
    val_y_rotate[i * 4 + 2][1] = 2
    
    val_x_rotate[i * 4 + 3][0] = np.rot90(np.rot90(np.rot90(val_x[i][0])))
    val_x_rotate[i * 4 + 3][1] = np.rot90(np.rot90(np.rot90(val_x[i][1])))
    val_x_rotate[i * 4 + 3][2] = np.rot90(np.rot90(np.rot90(val_x[i][2])))
    val_y_rotate[i * 4 + 3][0] = val_y[i]
    val_y_rotate[i * 4 + 3][1] = 3
    
for i in range(test_x.shape[0]):
    test_x_rotate[i * 4 + 0] = test_x[i]
    test_y_rotate[i * 4 + 0][0] = test_y[i]
    test_y_rotate[i * 4 + 0][1] = 0
    
    test_x_rotate[i * 4 + 1][0] = np.rot90(test_x[i][0])
    test_x_rotate[i * 4 + 1][1] = np.rot90(test_x[i][1])
    test_x_rotate[i * 4 + 1][2] = np.rot90(test_x[i][2])
    test_y_rotate[i * 4 + 1][0] = test_y[i]
    test_y_rotate[i * 4 + 1][1] = 1
    
    test_x_rotate[i * 4 + 2][0] = np.rot90(np.rot90(test_x[i][0]))
    test_x_rotate[i * 4 + 2][1] = np.rot90(np.rot90(test_x[i][1]))
    test_x_rotate[i * 4 + 2][2] = np.rot90(np.rot90(test_x[i][2]))
    test_y_rotate[i * 4 + 2][0] = test_y[i]
    test_y_rotate[i * 4 + 2][1] = 2
    
    test_x_rotate[i * 4 + 3][0] = np.rot90(np.rot90(np.rot90(test_x[i][0])))
    test_x_rotate[i * 4 + 3][1] = np.rot90(np.rot90(np.rot90(test_x[i][1])))
    test_x_rotate[i * 4 + 3][2] = np.rot90(np.rot90(np.rot90(test_x[i][2])))
    test_y_rotate[i * 4 + 3][0] = test_y[i]
    test_y_rotate[i * 4 + 3][1] = 3

In [15]:
print("test_x shape: ", test_x.shape)
print("test_x_rotate shape: ", test_x_rotate.shape)

test_x shape:  (12630, 3, 50, 50)
test_x_rotate shape:  (50520, 3, 50, 50)


In [16]:
test_x = np.swapaxes(test_x,3,1)
test_x_rotate = np.swapaxes(test_x_rotate,3,1)
test_x = np.swapaxes(test_x,2,1)
test_x_rotate = np.swapaxes(test_x_rotate,2,1)
print("test_x shape: ", test_x.shape)
print("test_x_rotate shape: ", test_x_rotate.shape)

test_x shape:  (12630, 50, 50, 3)
test_x_rotate shape:  (50520, 50, 50, 3)


In [17]:
test_y = torch.from_numpy(test_y)
test_y_rotate = torch.from_numpy(test_y_rotate)
print(test_y.size())
print(test_y_rotate.size())

torch.Size([12630])
torch.Size([50520, 2])


# prepare gauss noise

In [29]:
'''
def gauss_noisy(image):
  row,col,ch= image.shape
  mean = 0
  var = 40
  sigma = var**0.5
  gauss = np.random.normal(mean,sigma,(row,col,ch))
  gauss = gauss.reshape(row,col,ch)
  noisy = image + gauss
  return noisy
'''
def gauss_noisy(x, severity=1):
    c = [0.04, 0.06, .08, .09, .10][severity - 1]

    x = np.array(x) / 255.
    return np.clip(x + np.random.normal(size=x.shape, scale=c), 0, 1) * 255

In [30]:
test_x_gauss_noise = np.zeros(test_x.shape)
test_x_rotate_gauss_noise = np.zeros(test_x_rotate.shape)
print("test_x_gauss_noise shape: ", test_x_gauss_noise.shape)
print("test_x_rotate_gauss_noise shape: ", test_x_rotate_gauss_noise.shape)

test_x_gauss_noise shape:  (12630, 50, 50, 3)
test_x_rotate_gauss_noise shape:  (50520, 50, 50, 3)


In [31]:
for i in range(test_x.shape[0]):
    test_x_gauss_noise[i] = gauss_noisy(test_x[i])
    
for i in range(test_x_rotate.shape[0]):
    test_x_rotate_gauss_noise[i] = gauss_noisy(test_x_rotate[i])

In [32]:
cv2.imwrite("test_1370.png", test_x[1370])
cv2.imwrite("test_1371.png", test_x[1371])
cv2.imwrite("test_1372.png", test_x[1372])
cv2.imwrite("test_1373.png", test_x[1373])
cv2.imwrite("test_1374.png", test_x[1374])
cv2.imwrite("test_1375.png", test_x[1375])
cv2.imwrite("test_1376.png", test_x[1376])
cv2.imwrite("test_1377.png", test_x[1377])
cv2.imwrite("test_1378.png", test_x[1378])
cv2.imwrite("test_1379.png", test_x[1379])
cv2.imwrite("test_1370_gauss_noise.png", test_x_gauss_noise[1370])
cv2.imwrite("test_1371_gauss_noise.png", test_x_gauss_noise[1371])
cv2.imwrite("test_1372_gauss_noise.png", test_x_gauss_noise[1372])
cv2.imwrite("test_1373_gauss_noise.png", test_x_gauss_noise[1373])
cv2.imwrite("test_1374_gauss_noise.png", test_x_gauss_noise[1374])
cv2.imwrite("test_1375_gauss_noise.png", test_x_gauss_noise[1375])
cv2.imwrite("test_1376_gauss_noise.png", test_x_gauss_noise[1376])
cv2.imwrite("test_1377_gauss_noise.png", test_x_gauss_noise[1377])
cv2.imwrite("test_1378_gauss_noise.png", test_x_gauss_noise[1378])
cv2.imwrite("test_1379_gauss_noise.png", test_x_gauss_noise[1379])

True

In [22]:
test_x_gauss_noise = np.swapaxes(test_x_gauss_noise,3,1)
test_x_rotate_gauss_noise = np.swapaxes(test_x_rotate_gauss_noise,3,1)
test_x_gauss_noise = np.swapaxes(test_x_gauss_noise,2,3)
test_x_rotate_gauss_noise = np.swapaxes(test_x_rotate_gauss_noise,2,3)
print("test_x_gauss_noise shape: ", test_x_gauss_noise.shape)
print("test_x_rotate_gauss_noise shape: ", test_x_rotate_gauss_noise.shape)

test_x_gauss_noise shape:  (12630, 3, 50, 50)
test_x_rotate_gauss_noise shape:  (50520, 3, 50, 50)


In [23]:
# Compute mean and std.
train_mean = []
train_std = []
for i in range(3):
    s = np.std(train_x[:,i,:,:])
    m = np.mean(train_x[:,i,:,:])
    
    test_x_gauss_noise[:,i,:,:] = (test_x_gauss_noise[:,i,:,:] - m) / s
    test_x_rotate_gauss_noise[:,i,:,:] = (test_x_rotate_gauss_noise[:,i,:,:] - m) / s
    
    train_std.append(np.std(train_x[:,i,:,:]))
    train_mean.append(np.mean(train_x[:,i,:,:]))
print(f"Train mean: {train_mean}")
print(f"Train std: {train_std}")

Train mean: [87.565186, 79.653656, 82.58276]
Train std: [70.78099, 67.66874, 69.59343]


In [24]:
test_x_gauss_noise = torch.from_numpy(test_x_gauss_noise)
test_x_rotate_gauss_noise = torch.from_numpy(test_x_rotate_gauss_noise)
print(test_x_gauss_noise.size())
print(test_x_rotate_gauss_noise.size())

torch.Size([12630, 3, 50, 50])
torch.Size([50520, 3, 50, 50])


In [25]:

torch.save(test_x_gauss_noise, 'test_x_gauss_noise.pt')
torch.save(test_y, 'test_y_gauss_noise.pt')
torch.save(test_x_rotate_gauss_noise, 'test_x_rotate_gauss_noise.pt')
torch.save(test_y_rotate, 'test_y_rotate_gauss_noise.pt')

del test_x_gauss_noise
del test_x_rotate_gauss_noise



# prepare defocus blur

In [26]:
def disk(radius, alias_blur=0.1, dtype=np.float32):
    if radius <= 8:
        L = np.arange(-8, 8 + 1)
        ksize = (3, 3)
    else:
        L = np.arange(-radius, radius + 1)
        ksize = (5, 5)
    X, Y = np.meshgrid(L, L)
    aliased_disk = np.array((X ** 2 + Y ** 2) <= radius ** 2, dtype=dtype)
    aliased_disk /= np.sum(aliased_disk)

    # supersample disk to antialias
    return cv2.GaussianBlur(aliased_disk, ksize=ksize, sigmaX=alias_blur)

def defocus_blur(x, severity=5):
    c = [(0.3, 0.4), (0.4, 0.5), (0.5, 0.6), (1, 0.2), (1.5, 0.1)][severity - 1]

    x = np.array(x) / 255.
    kernel = disk(radius=c[0], alias_blur=c[1])

    channels = []
    for d in range(3):
        channels.append(cv2.filter2D(x[:, :, d], -1, kernel))
    channels = np.array(channels).transpose((1, 2, 0))  # 3x32x32 -> 32x32x3

    return np.clip(channels, 0, 1) * 255

In [27]:
test_x_defocus_blur = np.zeros(test_x.shape)
test_x_rotate_defocus_blur = np.zeros(test_x_rotate.shape)
print("test_x_defocus_blur shape: ", test_x_defocus_blur.shape)
print("test_x_rotate_defocus_blur shape: ", test_x_rotate_defocus_blur.shape)

test_x_defocus_blur shape:  (12630, 50, 50, 3)
test_x_rotate_defocus_blur shape:  (50520, 50, 50, 3)


In [28]:
for i in range(test_x.shape[0]):
    test_x_defocus_blur[i] = defocus_blur(test_x[i])
    
for i in range(test_x_rotate.shape[0]):
    test_x_rotate_defocus_blur[i] = defocus_blur(test_x_rotate[i])

In [29]:
cv2.imwrite("test_1370_defocus_blur.png", test_x_defocus_blur[1370])
cv2.imwrite("test_1371_defocus_blur.png", test_x_defocus_blur[1371])
cv2.imwrite("test_1372_defocus_blur.png", test_x_defocus_blur[1372])
cv2.imwrite("test_1373_defocus_blur.png", test_x_defocus_blur[1373])
cv2.imwrite("test_1374_defocus_blur.png", test_x_defocus_blur[1374])
cv2.imwrite("test_1375_defocus_blur.png", test_x_defocus_blur[1375])
cv2.imwrite("test_1376_defocus_blur.png", test_x_defocus_blur[1376])
cv2.imwrite("test_1377_defocus_blur.png", test_x_defocus_blur[1377])
cv2.imwrite("test_1378_defocus_blur.png", test_x_defocus_blur[1378])
cv2.imwrite("test_1379_defocus_blur.png", test_x_defocus_blur[1379])

True

In [30]:
test_x_defocus_blur = np.swapaxes(test_x_defocus_blur,3,1)
test_x_rotate_defocus_blur = np.swapaxes(test_x_rotate_defocus_blur,3,1)
test_x_defocus_blur = np.swapaxes(test_x_defocus_blur,2,3)
test_x_rotate_defocus_blur = np.swapaxes(test_x_rotate_defocus_blur,2,3)
print("test_x_defocus_blur shape: ", test_x_defocus_blur.shape)
print("test_x_rotate_defocus_blur shape: ", test_x_rotate_defocus_blur.shape)

test_x_defocus_blur shape:  (12630, 3, 50, 50)
test_x_rotate_defocus_blur shape:  (50520, 3, 50, 50)


In [31]:
# Compute mean and std.
train_mean = []
train_std = []
for i in range(3):
    s = np.std(train_x[:,i,:,:])
    m = np.mean(train_x[:,i,:,:])
    
    test_x_defocus_blur[:,i,:,:] = (test_x_defocus_blur[:,i,:,:] - m) / s
    test_x_rotate_defocus_blur[:,i,:,:] = (test_x_rotate_defocus_blur[:,i,:,:] - m) / s
    
    train_std.append(np.std(train_x[:,i,:,:]))
    train_mean.append(np.mean(train_x[:,i,:,:]))
print(f"Train mean: {train_mean}")
print(f"Train std: {train_std}")

Train mean: [87.565186, 79.653656, 82.58276]
Train std: [70.78099, 67.66874, 69.59343]


In [32]:
test_x_defocus_blur = torch.from_numpy(test_x_defocus_blur)
test_x_rotate_defocus_blur = torch.from_numpy(test_x_rotate_defocus_blur)
print(test_x_defocus_blur.size())
print(test_x_rotate_defocus_blur.size())

torch.Size([12630, 3, 50, 50])
torch.Size([50520, 3, 50, 50])


In [33]:

torch.save(test_x_defocus_blur, 'test_x_defocus_blur.pt')
torch.save(test_y, 'test_y_defocus_blur.pt')
torch.save(test_x_rotate_defocus_blur, 'test_x_rotate_defocus_blur.pt')
torch.save(test_y_rotate, 'test_y_rotate_defocus_blur.pt')

del test_x_defocus_blur
del test_x_rotate_defocus_blur


# prepare zoom blur

In [34]:
from scipy.ndimage import zoom as scizoom
def clipped_zoom(img, zoom_factor):
    h = img.shape[0]
    # ceil crop height(= crop width)
    ch = int(np.ceil(h / zoom_factor))

    top = (h - ch) // 2
    img = scizoom(img[top:top + ch, top:top + ch], (zoom_factor, zoom_factor, 1), order=1)
    # trim off any extra pixels
    trim_top = (img.shape[0] - h) // 2

    return img[trim_top:trim_top + h, trim_top:trim_top + h]

def zoom_blur(x, severity=5):
    c = [np.arange(1, 1.06, 0.01), np.arange(1, 1.11, 0.01), np.arange(1, 1.16, 0.01),
         np.arange(1, 1.21, 0.01), np.arange(1, 1.26, 0.01)][severity - 1]

    x = (np.array(x) / 255.).astype(np.float32)
    out = np.zeros_like(x)
    for zoom_factor in c:
        out += clipped_zoom(x, zoom_factor)

    x = (x + out) / (len(c) + 1)
    return np.clip(x, 0, 1) * 255

In [35]:
test_x_zoom_blur = np.zeros(test_x.shape)
test_x_rotate_zoom_blur = np.zeros(test_x_rotate.shape)
print("test_x_zoom_blur shape: ", test_x_zoom_blur.shape)
print("test_x_rotate_zoom_blur shape: ", test_x_rotate_zoom_blur.shape)

test_x_zoom_blur shape:  (12630, 50, 50, 3)
test_x_rotate_zoom_blur shape:  (50520, 50, 50, 3)


In [36]:
for i in range(test_x.shape[0]):
    test_x_zoom_blur[i] = zoom_blur(test_x[i])
    
for i in range(test_x_rotate.shape[0]):
    test_x_rotate_zoom_blur[i] = zoom_blur(test_x_rotate[i])

In [37]:
cv2.imwrite("test_1370_zoom_blur.png", test_x_zoom_blur[1370])
cv2.imwrite("test_1371_zoom_blur.png", test_x_zoom_blur[1371])
cv2.imwrite("test_1372_zoom_blur.png", test_x_zoom_blur[1372])
cv2.imwrite("test_1373_zoom_blur.png", test_x_zoom_blur[1373])
cv2.imwrite("test_1374_zoom_blur.png", test_x_zoom_blur[1374])
cv2.imwrite("test_1375_zoom_blur.png", test_x_zoom_blur[1375])
cv2.imwrite("test_1376_zoom_blur.png", test_x_zoom_blur[1376])
cv2.imwrite("test_1377_zoom_blur.png", test_x_zoom_blur[1377])
cv2.imwrite("test_1378_zoom_blur.png", test_x_zoom_blur[1378])
cv2.imwrite("test_1379_zoom_blur.png", test_x_zoom_blur[1379])

True

In [38]:
test_x_zoom_blur = np.swapaxes(test_x_zoom_blur,3,1)
test_x_rotate_zoom_blur = np.swapaxes(test_x_rotate_zoom_blur,3,1)
test_x_zoom_blur = np.swapaxes(test_x_zoom_blur,2,3)
test_x_rotate_zoom_blur = np.swapaxes(test_x_rotate_zoom_blur,2,3)
print("test_x_zoom_blur shape: ", test_x_zoom_blur.shape)
print("test_x_rotate_zoom_blur shape: ", test_x_rotate_zoom_blur.shape)

test_x_zoom_blur shape:  (12630, 3, 50, 50)
test_x_rotate_zoom_blur shape:  (50520, 3, 50, 50)


In [39]:
# Compute mean and std.
train_mean = []
train_std = []
for i in range(3):
    s = np.std(train_x[:,i,:,:])
    m = np.mean(train_x[:,i,:,:])
    
    test_x_zoom_blur[:,i,:,:] = (test_x_zoom_blur[:,i,:,:] - m) / s
    test_x_rotate_zoom_blur[:,i,:,:] = (test_x_rotate_zoom_blur[:,i,:,:] - m) / s
    
    train_std.append(np.std(train_x[:,i,:,:]))
    train_mean.append(np.mean(train_x[:,i,:,:]))
print(f"Train mean: {train_mean}")
print(f"Train std: {train_std}")

Train mean: [87.565186, 79.653656, 82.58276]
Train std: [70.78099, 67.66874, 69.59343]


In [40]:
test_x_zoom_blur = torch.from_numpy(test_x_zoom_blur)
test_x_rotate_zoom_blur = torch.from_numpy(test_x_rotate_zoom_blur)
print(test_x_zoom_blur.size())
print(test_x_rotate_zoom_blur.size())

torch.Size([12630, 3, 50, 50])
torch.Size([50520, 3, 50, 50])


In [41]:

torch.save(test_x_zoom_blur, 'test_x_zoom_blur.pt')
torch.save(test_y, 'test_y_zoom_blur.pt')
torch.save(test_x_rotate_zoom_blur, 'test_x_rotate_zoom_blur.pt')
torch.save(test_y_rotate, 'test_y_rotate_zoom_blur.pt')

del test_x_zoom_blur
del test_x_rotate_zoom_blur


# prepare horizontal motion blur

In [42]:
# applying the kernel to the input image
def hmotion_blur(img):
    # Specify the kernel size. 
    # The greater the size, the more the motion. 
    kernel_size = 10

    # Create the vertical kernel. 
    kernel_v = np.zeros((kernel_size, kernel_size)) 

    # Create a copy of the same for creating the horizontal kernel. 
    kernel_h = np.copy(kernel_v) 

    # Fill the middle row with ones. 
    #kernel_v[:, int((kernel_size - 1)/2)] = np.ones(kernel_size) 
    kernel_h[int((kernel_size - 1)/2), :] = np.ones(kernel_size) 

    # Normalize. 
    #kernel_v /= kernel_size 
    kernel_h /= kernel_size 

    # Apply the vertical kernel. 
    #vertical_mb = cv2.filter2D(img, -1, kernel_v) 

    # Apply the horizontal kernel. 
    horizonal_mb = cv2.filter2D(img, -1, kernel_h) 
    return horizonal_mb

In [43]:
test_x_hmotion_blur = np.zeros(test_x.shape)
test_x_rotate_hmotion_blur = np.zeros(test_x_rotate.shape)
print("test_x_hmotion_blur shape: ", test_x_hmotion_blur.shape)
print("test_x_rotate_hmotion_blur shape: ", test_x_rotate_hmotion_blur.shape)

test_x_motion_blur shape:  (12630, 50, 50, 3)
test_x_rotate_motion_blur shape:  (50520, 50, 50, 3)


In [44]:
for i in range(test_x.shape[0]):
    test_x_hmotion_blur[i] = hmotion_blur(test_x[i])
    
for i in range(test_x_rotate.shape[0]):
    test_x_rotate_hmotion_blur[i] = hmotion_blur(test_x_rotate[i])

In [45]:
cv2.imwrite("test_1370_hmotion_blur.png", test_x_hmotion_blur[1370])
cv2.imwrite("test_1371_hmotion_blur.png", test_x_hmotion_blur[1371])
cv2.imwrite("test_1372_hmotion_blur.png", test_x_hmotion_blur[1372])
cv2.imwrite("test_1373_hmotion_blur.png", test_x_hmotion_blur[1373])
cv2.imwrite("test_1374_hmotion_blur.png", test_x_hmotion_blur[1374])
cv2.imwrite("test_1375_hmotion_blur.png", test_x_hmotion_blur[1375])
cv2.imwrite("test_1376_hmotion_blur.png", test_x_hmotion_blur[1376])
cv2.imwrite("test_1377_hmotion_blur.png", test_x_hmotion_blur[1377])
cv2.imwrite("test_1378_hmotion_blur.png", test_x_hmotion_blur[1378])
cv2.imwrite("test_1379_hmotion_blur.png", test_x_hmotion_blur[1379])

True

In [46]:
test_x_hmotion_blur = np.swapaxes(test_x_hmotion_blur,3,1)
test_x_rotate_hmotion_blur = np.swapaxes(test_x_rotate_hmotion_blur,3,1)
test_x_hmotion_blur = np.swapaxes(test_x_hmotion_blur,2,3)
test_x_rotate_hmotion_blur = np.swapaxes(test_x_rotate_hmotion_blur,2,3)
print("test_x_hmotion_blur shape: ", test_x_hmotion_blur.shape)
print("test_x_rotate_hmotion_blur shape: ", test_x_rotate_hmotion_blur.shape)

test_x_motion_blur shape:  (12630, 3, 50, 50)
test_x_rotate_motion_blur shape:  (50520, 3, 50, 50)


In [47]:
# Compute mean and std.
train_mean = []
train_std = []
for i in range(3):
    s = np.std(train_x[:,i,:,:])
    m = np.mean(train_x[:,i,:,:])
    
    test_x_hmotion_blur[:,i,:,:] = (test_x_hmotion_blur[:,i,:,:] - m) / s
    test_x_rotate_hmotion_blur[:,i,:,:] = (test_x_rotate_hmotion_blur[:,i,:,:] - m) / s
    
    train_std.append(np.std(train_x[:,i,:,:]))
    train_mean.append(np.mean(train_x[:,i,:,:]))
print(f"Train mean: {train_mean}")
print(f"Train std: {train_std}")

Train mean: [87.565186, 79.653656, 82.58276]
Train std: [70.78099, 67.66874, 69.59343]


In [48]:
test_x_hmotion_blur = torch.from_numpy(test_x_hmotion_blur)
test_x_rotate_hmotion_blur = torch.from_numpy(test_x_rotate_hmotion_blur)
print(test_x_hmotion_blur.size())
print(test_x_rotate_hmotion_blur.size())

torch.Size([12630, 3, 50, 50])
torch.Size([50520, 3, 50, 50])


In [49]:

torch.save(test_x_hmotion_blur, 'test_x_hmotion_blur.pt')
torch.save(test_y, 'test_y_hmotion_blur.pt')
torch.save(test_x_rotate_hmotion_blur, 'test_x_rotate_hmotion_blur.pt')
torch.save(test_y_rotate, 'test_y_rotate_hmotion_blur.pt')

del test_x_motion_blur
del test_x_rotate_motion_blur


# prepare vertical motion blur

In [ ]:
# applying the kernel to the input image
def vmotion_blur(img):
    # Specify the kernel size. 
    # The greater the size, the more the motion. 
    kernel_size = 10

    # Create the vertical kernel. 
    kernel_v = np.zeros((kernel_size, kernel_size)) 

    # Create a copy of the same for creating the horizontal kernel. 
    kernel_h = np.copy(kernel_v) 

    # Fill the middle row with ones. 
    kernel_v[:, int((kernel_size - 1)/2)] = np.ones(kernel_size) 
    kernel_h[int((kernel_size - 1)/2), :] = np.ones(kernel_size) 

    # Normalize. 
    kernel_v /= kernel_size 
    kernel_h /= kernel_size 

    # Apply the vertical kernel. 
    vertical_mb = cv2.filter2D(img, -1, kernel_v) 

    # Apply the horizontal kernel. 
    #horizonal_mb = cv2.filter2D(img, -1, kernel_h) 
    return vertical_mb

In [ ]:
test_x_vmotion_blur = np.zeros(test_x.shape)
test_x_rotate_vmotion_blur = np.zeros(test_x_rotate.shape)
print("test_x_vmotion_blur shape: ", test_x_vmotion_blur.shape)
print("test_x_rotate_vmotion_blur shape: ", test_x_rotate_vmotion_blur.shape)

In [ ]:
for i in range(test_x.shape[0]):
    test_x_vmotion_blur[i] = vmotion_blur(test_x[i])
    
for i in range(test_x_rotate.shape[0]):
    test_x_rotate_vmotion_blur[i] = vmotion_blur(test_x_rotate[i])

In [ ]:
cv2.imwrite("test_1370_vmotion_blur.png", test_x_vmotion_blur[1370])
cv2.imwrite("test_1371_vmotion_blur.png", test_x_vmotion_blur[1371])
cv2.imwrite("test_1372_vmotion_blur.png", test_x_vmotion_blur[1372])
cv2.imwrite("test_1373_vmotion_blur.png", test_x_vmotion_blur[1373])
cv2.imwrite("test_1374_vmotion_blur.png", test_x_vmotion_blur[1374])
cv2.imwrite("test_1375_vmotion_blur.png", test_x_vmotion_blur[1375])
cv2.imwrite("test_1376_vmotion_blur.png", test_x_vmotion_blur[1376])
cv2.imwrite("test_1377_vmotion_blur.png", test_x_vmotion_blur[1377])
cv2.imwrite("test_1378_vmotion_blur.png", test_x_vmotion_blur[1378])
cv2.imwrite("test_1379_vmotion_blur.png", test_x_vmotion_blur[1379])

In [ ]:
test_x_vmotion_blur = np.swapaxes(test_x_vmotion_blur,3,1)
test_x_rotate_vmotion_blur = np.swapaxes(test_x_rotate_vmotion_blur,3,1)
test_x_vmotion_blur = np.swapaxes(test_x_vmotion_blur,2,3)
test_x_rotate_vmotion_blur = np.swapaxes(test_x_rotate_vmotion_blur,2,3)
print("test_x_vmotion_blur shape: ", test_x_vmotion_blur.shape)
print("test_x_rotate_vmotion_blur shape: ", test_x_rotate_vmotion_blur.shape)

In [ ]:
# Compute mean and std.
train_mean = []
train_std = []
for i in range(3):
    s = np.std(train_x[:,i,:,:])
    m = np.mean(train_x[:,i,:,:])
    
    test_x_vmotion_blur[:,i,:,:] = (test_x_vmotion_blur[:,i,:,:] - m) / s
    test_x_rotate_vmotion_blur[:,i,:,:] = (test_x_rotate_vmotion_blur[:,i,:,:] - m) / s
    
    train_std.append(np.std(train_x[:,i,:,:]))
    train_mean.append(np.mean(train_x[:,i,:,:]))
print(f"Train mean: {train_mean}")
print(f"Train std: {train_std}")

In [ ]:
test_x_vmotion_blur = torch.from_numpy(test_x_vmotion_blur)
test_x_rotate_vmotion_blur = torch.from_numpy(test_x_rotate_vmotion_blur)
print(test_x_vmotion_blur.size())
print(test_x_rotate_vmotion_blur.size())

In [ ]:

torch.save(test_x_vmotion_blur, 'test_x_vmotion_blur.pt')
torch.save(test_y, 'test_y_vmotion_blur.pt')
torch.save(test_x_rotate_vmotion_blur, 'test_x_rotate_vmotion_blur.pt')
torch.save(test_y_rotate, 'test_y_rotate_vmotion_blur.pt')

del test_x_motion_blur
del test_x_rotate_motion_blur


# prepare Elastic Transform

In [50]:
from skimage.filters import gaussian
from scipy.ndimage.interpolation import map_coordinates
def elastic_transform(image, severity=5):
    IMSIZE = 32
    c = [(IMSIZE*0, IMSIZE*0, IMSIZE*0.08),
         (IMSIZE*0.05, IMSIZE*0.2, IMSIZE*0.07),
         (IMSIZE*0.08, IMSIZE*0.06, IMSIZE*0.06),
         (IMSIZE*0.1, IMSIZE*0.04, IMSIZE*0.05),
         (IMSIZE*0.1, IMSIZE*0.03, IMSIZE*0.03)][severity - 1]

    image = np.array(image, dtype=np.float32) / 255.
    shape = image.shape
    shape_size = shape[:2]

    # random affine
    center_square = np.float32(shape_size) // 2
    square_size = min(shape_size) // 3
    pts1 = np.float32([center_square + square_size,
                       [center_square[0] + square_size, center_square[1] - square_size],
                       center_square - square_size])
    pts2 = pts1 + np.random.uniform(-c[2], c[2], size=pts1.shape).astype(np.float32)
    M = cv2.getAffineTransform(pts1, pts2)
    image = cv2.warpAffine(image, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)

    dx = (gaussian(np.random.uniform(-1, 1, size=shape[:2]),
                   c[1], mode='reflect', truncate=3) * c[0]).astype(np.float32)
    dy = (gaussian(np.random.uniform(-1, 1, size=shape[:2]),
                   c[1], mode='reflect', truncate=3) * c[0]).astype(np.float32)
    dx, dy = dx[..., np.newaxis], dy[..., np.newaxis]

    x, y, z = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]), np.arange(shape[2]))
    indices = np.reshape(y + dy, (-1, 1)), np.reshape(x + dx, (-1, 1)), np.reshape(z, (-1, 1))
    return np.clip(map_coordinates(image, indices, order=1, mode='reflect').reshape(shape), 0, 1) * 255

In [51]:
test_x_elastic_transform = np.zeros(test_x.shape)
test_x_rotate_elastic_transform = np.zeros(test_x_rotate.shape)
print("test_x_elastic_transform shape: ", test_x_elastic_transform.shape)
print("test_x_rotate_elastic_transform shape: ", test_x_rotate_elastic_transform.shape)

test_x_elastic_transform shape:  (12630, 50, 50, 3)
test_x_rotate_elastic_transform shape:  (50520, 50, 50, 3)


In [52]:
for i in range(test_x.shape[0]):
    test_x_elastic_transform[i] = elastic_transform(test_x[i])
    
for i in range(test_x_rotate.shape[0]):
    test_x_rotate_elastic_transform[i] = elastic_transform(test_x_rotate[i])

In [53]:
cv2.imwrite("test_1370_elastic_transform.png", test_x_elastic_transform[1370])
cv2.imwrite("test_1371_elastic_transform.png", test_x_elastic_transform[1371])
cv2.imwrite("test_1372_elastic_transform.png", test_x_elastic_transform[1372])
cv2.imwrite("test_1373_elastic_transform.png", test_x_elastic_transform[1373])
cv2.imwrite("test_1374_elastic_transform.png", test_x_elastic_transform[1374])
cv2.imwrite("test_1375_elastic_transform.png", test_x_elastic_transform[1375])
cv2.imwrite("test_1376_elastic_transform.png", test_x_elastic_transform[1376])
cv2.imwrite("test_1377_elastic_transform.png", test_x_elastic_transform[1377])
cv2.imwrite("test_1378_elastic_transform.png", test_x_elastic_transform[1378])
cv2.imwrite("test_1379_elastic_transform.png", test_x_elastic_transform[1379])

True

In [54]:
test_x_elastic_transform = np.swapaxes(test_x_elastic_transform,3,1)
test_x_rotate_elastic_transform = np.swapaxes(test_x_rotate_elastic_transform,3,1)
test_x_elastic_transform = np.swapaxes(test_x_elastic_transform,2,3)
test_x_rotate_elastic_transform = np.swapaxes(test_x_rotate_elastic_transform,2,3)
print("test_x_elastic_transform shape: ", test_x_elastic_transform.shape)
print("test_x_rotate_elastic_transform shape: ", test_x_rotate_elastic_transform.shape)

test_x_elastic_transform shape:  (12630, 3, 50, 50)
test_x_rotate_elastic_transform shape:  (50520, 3, 50, 50)


In [55]:
# Compute mean and std.
train_mean = []
train_std = []
for i in range(3):
    s = np.std(train_x[:,i,:,:])
    m = np.mean(train_x[:,i,:,:])
    
    test_x_elastic_transform[:,i,:,:] = (test_x_elastic_transform[:,i,:,:] - m) / s
    test_x_rotate_elastic_transform[:,i,:,:] = (test_x_rotate_elastic_transform[:,i,:,:] - m) / s
    
    train_std.append(np.std(train_x[:,i,:,:]))
    train_mean.append(np.mean(train_x[:,i,:,:]))
print(f"Train mean: {train_mean}")
print(f"Train std: {train_std}")

Train mean: [87.565186, 79.653656, 82.58276]
Train std: [70.78099, 67.66874, 69.59343]


In [56]:
test_x_elastic_transform = torch.from_numpy(test_x_elastic_transform)
test_x_rotate_elastic_transform = torch.from_numpy(test_x_rotate_elastic_transform)
print(test_x_elastic_transform.size())
print(test_x_rotate_elastic_transform.size())

torch.Size([12630, 3, 50, 50])
torch.Size([50520, 3, 50, 50])


In [57]:

torch.save(test_x_elastic_transform, 'test_x_elastic_transform.pt')
torch.save(test_y, 'test_y_elastic_transform.pt')
torch.save(test_x_rotate_elastic_transform, 'test_x_rotate_elastic_transform.pt')
torch.save(test_y_rotate, 'test_y_rotate_elastic_transform.pt')

del test_x_elastic_transform
del test_x_rotate_elastic_transform


# prepare Contrast

In [20]:
def contrast(x, severity=5):
    c = [.75, .5, .4, .3, 0.15][severity - 1]

    x = np.array(x) / 255.
    means = np.mean(x, axis=(0, 1), keepdims=True)
    return np.clip((x - means) * c + means, 0, 1) * 255

In [18]:
test_x_contrast = np.zeros(test_x.shape)
test_x_rotate_contrast = np.zeros(test_x_rotate.shape)
print("test_x_contrast shape: ", test_x_contrast.shape)
print("test_x_rotate_contrast shape: ", test_x_rotate_contrast.shape)

test_x_contrast shape:  (12630, 50, 50, 3)
test_x_rotate_contrast shape:  (50520, 50, 50, 3)


In [21]:
for i in range(test_x.shape[0]):
    test_x_contrast[i] = contrast(test_x[i])
    
for i in range(test_x_rotate.shape[0]):
    test_x_rotate_contrast[i] = contrast(test_x_rotate[i])

In [22]:
cv2.imwrite("test_1370_contrast.png", test_x_contrast[1370])
cv2.imwrite("test_1371_contrast.png", test_x_contrast[1371])
cv2.imwrite("test_1372_contrast.png", test_x_contrast[1372])
cv2.imwrite("test_1373_contrast.png", test_x_contrast[1373])
cv2.imwrite("test_1374_contrast.png", test_x_contrast[1374])
cv2.imwrite("test_1375_contrast.png", test_x_contrast[1375])
cv2.imwrite("test_1376_contrast.png", test_x_contrast[1376])
cv2.imwrite("test_1377_contrast.png", test_x_contrast[1377])
cv2.imwrite("test_1378_contrast.png", test_x_contrast[1378])
cv2.imwrite("test_1379_contrast.png", test_x_contrast[1379])

True

In [23]:
test_x_contrast = np.swapaxes(test_x_contrast,3,1)
test_x_rotate_contrast = np.swapaxes(test_x_rotate_contrast,3,1)
test_x_contrast = np.swapaxes(test_x_contrast,2,3)
test_x_rotate_contrast = np.swapaxes(test_x_rotate_contrast,2,3)
print("test_x_contrast shape: ", test_x_contrast.shape)
print("test_x_rotate_contrast shape: ", test_x_rotate_contrast.shape)

test_x_contrast shape:  (12630, 3, 50, 50)
test_x_rotate_contrast shape:  (50520, 3, 50, 50)


In [ ]:
# Compute mean and std.
train_mean = []
train_std = []
for i in range(3):
    s = np.std(train_x[:,i,:,:])
    m = np.mean(train_x[:,i,:,:])
    
    test_x_contrast[:,i,:,:] = (test_x_contrast[:,i,:,:] - m) / s
    test_x_rotate_contrast[:,i,:,:] = (test_x_rotate_contrast[:,i,:,:] - m) / s
    
    train_std.append(np.std(train_x[:,i,:,:]))
    train_mean.append(np.mean(train_x[:,i,:,:]))
print(f"Train mean: {train_mean}")
print(f"Train std: {train_std}")

In [ ]:
test_x_contrast = torch.from_numpy(test_x_contrast)
test_x_rotate_contrast = torch.from_numpy(test_x_rotate_contrast)
print(test_x_contrast.size())
print(test_x_rotate_contrast.size())

In [ ]:

torch.save(test_x_contrast, 'test_x_contrast.pt')
torch.save(test_y, 'test_y_contrast.pt')
torch.save(test_x_rotate_contrast, 'test_x_rotate_contrast.pt')
torch.save(test_y_rotate, 'test_y_contrast.pt')

del test_x_contrast
del test_x_rotate_contrast


# prepare Shot noise

In [24]:
def shot_noise(x, severity=5):
    c = [500, 250, 100, 75, 50][severity - 1]

    x = np.array(x) / 255.
    return np.clip(np.random.poisson(x * c) / c, 0, 1) * 255

In [25]:
test_x_shot = np.zeros(test_x.shape)
test_x_rotate_shot = np.zeros(test_x_rotate.shape)
print("test_x_shot shape: ", test_x_shot.shape)
print("test_x_rotate_shot shape: ", test_x_rotate_shot.shape)

test_x_shot shape:  (12630, 50, 50, 3)
test_x_rotate_shot shape:  (50520, 50, 50, 3)


In [26]:
for i in range(test_x.shape[0]):
    test_x_shot[i] = shot_noise(test_x[i])
    
for i in range(test_x_rotate.shape[0]):
    test_x_rotate_shot[i] = shot_noise(test_x_rotate[i])

In [27]:
cv2.imwrite("test_1370_shot.png", test_x_shot[1370])
cv2.imwrite("test_1371_shot.png", test_x_shot[1371])
cv2.imwrite("test_1372_shot.png", test_x_shot[1372])
cv2.imwrite("test_1373_shot.png", test_x_shot[1373])
cv2.imwrite("test_1374_shot.png", test_x_shot[1374])
cv2.imwrite("test_1375_shot.png", test_x_shot[1375])
cv2.imwrite("test_1376_shot.png", test_x_shot[1376])
cv2.imwrite("test_1377_shot.png", test_x_shot[1377])
cv2.imwrite("test_1378_shot.png", test_x_shot[1378])
cv2.imwrite("test_1379_shot.png", test_x_shot[1379])

True

In [28]:
test_x_shot = np.swapaxes(test_x_shot,3,1)
test_x_rotate_shot = np.swapaxes(test_x_rotate_shot,3,1)
test_x_shot = np.swapaxes(test_x_shot,2,3)
test_x_rotate_shot = np.swapaxes(test_x_rotate_shot,2,3)
print("test_x_shot shape: ", test_x_shot.shape)
print("test_x_rotate_shot shape: ", test_x_rotate_shot.shape)

test_x_shot shape:  (12630, 3, 50, 50)
test_x_rotate_shot shape:  (50520, 3, 50, 50)


In [ ]:
# Compute mean and std.
train_mean = []
train_std = []
for i in range(3):
    s = np.std(train_x[:,i,:,:])
    m = np.mean(train_x[:,i,:,:])
    
    test_x_shot[:,i,:,:] = (test_x_shot[:,i,:,:] - m) / s
    test_x_rotate_shot[:,i,:,:] = (test_x_rotate_shot[:,i,:,:] - m) / s
    
    train_std.append(np.std(train_x[:,i,:,:]))
    train_mean.append(np.mean(train_x[:,i,:,:]))
print(f"Train mean: {train_mean}")
print(f"Train std: {train_std}")

In [ ]:
test_x_shot = torch.from_numpy(test_x_shot)
test_x_rotate_shot = torch.from_numpy(test_x_rotate_shot)
print(test_x_shot.size())
print(test_x_rotate_shot.size())

In [ ]:

torch.save(test_x_shot, 'test_x_shot.pt')
torch.save(test_y, 'test_y_shot.pt')
torch.save(test_x_rotate_shot, 'test_x_rotate_shot.pt')
torch.save(test_y_rotate, 'test_y_shot.pt')

del test_x_shot
del test_x_rotate_shot


# prepare impulse noise

In [33]:
import skimage as sk
def impulse_noise(x, severity=5):
    c = [.01, .02, .03, .05, .07][severity - 1]

    x = sk.util.random_noise(np.array(x) / 255., mode='s&p', amount=c)
    return np.clip(x, 0, 1) * 255

In [34]:
test_x_impulse = np.zeros(test_x.shape)
test_x_rotate_impulse = np.zeros(test_x_rotate.shape)
print("test_x_impulse shape: ", test_x_impulse.shape)
print("test_x_rotate_impulse shape: ", test_x_rotate_impulse.shape)

test_x_impulse shape:  (12630, 50, 50, 3)
test_x_rotate_impulse shape:  (50520, 50, 50, 3)


In [35]:
for i in range(test_x.shape[0]):
    test_x_impulse[i] = impulse_noise(test_x[i])
    
for i in range(test_x_rotate.shape[0]):
    test_x_rotate_impulse[i] = impulse_noise(test_x_rotate[i])

In [36]:
cv2.imwrite("test_1370_impulse.png", test_x_impulse[1370])
cv2.imwrite("test_1371_impulse.png", test_x_impulse[1371])
cv2.imwrite("test_1372_impulse.png", test_x_impulse[1372])
cv2.imwrite("test_1373_impulse.png", test_x_impulse[1373])
cv2.imwrite("test_1374_impulse.png", test_x_impulse[1374])
cv2.imwrite("test_1375_impulse.png", test_x_impulse[1375])
cv2.imwrite("test_1376_impulse.png", test_x_impulse[1376])
cv2.imwrite("test_1377_impulse.png", test_x_impulse[1377])
cv2.imwrite("test_1378_impulse.png", test_x_impulse[1378])
cv2.imwrite("test_1379_impulse.png", test_x_impulse[1379])

True

In [37]:
test_x_impulse = np.swapaxes(test_x_impulse,3,1)
test_x_rotate_impulse = np.swapaxes(test_x_rotate_impulse,3,1)
test_x_impulse = np.swapaxes(test_x_impulse,2,3)
test_x_rotate_impulse = np.swapaxes(test_x_rotate_impulse,2,3)
print("test_x_impulse shape: ", test_x_impulse.shape)
print("test_x_rotate_impulse shape: ", test_x_rotate_impulse.shape)

test_x_impulse shape:  (12630, 3, 50, 50)
test_x_rotate_impulse shape:  (50520, 3, 50, 50)


In [ ]:
# Compute mean and std.
train_mean = []
train_std = []
for i in range(3):
    s = np.std(train_x[:,i,:,:])
    m = np.mean(train_x[:,i,:,:])
    
    test_x_impulse[:,i,:,:] = (test_x_impulse[:,i,:,:] - m) / s
    test_x_rotate_impulse[:,i,:,:] = (test_x_rotate_impulse[:,i,:,:] - m) / s
    
    train_std.append(np.std(train_x[:,i,:,:]))
    train_mean.append(np.mean(train_x[:,i,:,:]))
print(f"Train mean: {train_mean}")
print(f"Train std: {train_std}")

In [ ]:
test_x_impulse = torch.from_numpy(test_x_impulse)
test_x_rotate_impulse = torch.from_numpy(test_x_rotate_impulse)
print(test_x_impulse.size())
print(test_x_rotate_impulse.size())

In [ ]:

torch.save(test_x_impulse, 'test_x_impulse.pt')
torch.save(test_y, 'test_y_impulse.pt')
torch.save(test_x_rotate_impulse, 'test_x_rotate_impulse.pt')
torch.save(test_y_rotate, 'test_y_impulse.pt')

del test_x_impulse
del test_x_rotate_impulse
